# Extract and align data from Onix, Harp, Sleap, and photometry
## Cohort 1 and 2 working, Cohort 0: onix_digital Clock column is 0, explore why and/or use timestamps instead 

In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
#initiate variables 
has_heartbeat = False
cohort0 = False
cohort2 = False
onix_analog_clock_downsampled = False
onix_analog_framecount_upsampled = False
common_resampled_rate = 10000 #in Hz
unit_conversions = False #
save_full_asynchronous_data = False #saves alldata before resampling
has_photometry = True #if photometry data is present
has_onix_harp = True
has_video = True

#Cohort 2 tests for Encoder Test, NO photometry, No videodata1 
data_path = Path('/Users/rancze/Documents/Data/vestVR/EncoderTest/2025-02-26T15-03-34') #with ball rotations after motor rotation, first in Y, then in X
#data_path = Path('/Users/rancze/Documents/Data/vestVR/EncoderTest/2025-02-26T14-57-04')
has_heartbeat = True
has_photometry = False
has_onix_harp = False
has_video = False


photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

#create loaders 
session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.VideoReader("VideoData1")
video_reader2 = utils.VideoReader("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

### Load all data 

In [ ]:
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)

if has_photometry:
    print ("Loading processed photometry")
    photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
    photometry_data.set_index("TimeStamp", inplace=True)
    photometry_data.index.name = 'Seconds'
    print ("Loading processed photometry info")
    photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
    print ("Loading processed photometry events")
    photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')
    photometry_events["TimeStamp"] = photometry_events["TimeStamp"] /1000 # convert to seconds from ms
    photometry_events.set_index("TimeStamp", inplace=True)
    photometry_events.index.name = 'Seconds'

if has_video:
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data
if has_onix_harp:
    print ("Loading OnixDigital")
    onix_digital = utils.load_2(onix_digital_reader, data_path)

if cohort0:
    print ("Loading OnixAnalogFrameClock")
    onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
    
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData and converting to boolean photodiode array")
photodiode = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True, method='adaptive', refractory = 300, flip=True, verbose=False) #method adaptive or threshold (which is hard threshold at 120), refractory to avoid multiple detections

#read HARP data
print ("Loading H1 and H2 streams, AnalogInput removed")
harp_streams = utils.load_registers(data_path, dataframe = True, has_heartbeat = has_heartbeat, verbose = False) #loads as df, or if False, as dict
harp_streams.drop(columns=["AnalogInput(39)"], inplace=True)  # Removes AnalogInput permanently, as not currently used
harp_streams = harp_streams.dropna(how="all") # remove rows with all NaNs
# Convert specific columns in harp_streams to boolean type
columns_to_convert = []
for col in columns_to_convert:
    harp_streams[col] = harp_streams[col].astype(bool)

#read syncronising signal between HARP and ONIX
if not cohort0 and has_onix_harp:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )
    onix_harp["HarpTime"] = onix_harp["HarpTime"] + 1 # known issue with current version of ONIX, harp timestamps lag 1 second
    print ("❗Reminder: HarpTime was increased by 1s to account for know issue with ONIX")

print ("✅ Done Loading")

Convert platform position and flow sensor streams to real world units and forward fill 

In [ ]:
harp_streams_orig = harp_streams.copy() #keep original data for later use

In [ ]:
# Get encoder values for homing and next event positions as absolute real life 0 position 
homing_position, next_event_position = process.get_encoder_home_position(experiment_events, harp_streams)
print ("Encoder values for homing and next event positions")
print(f"Encoder value at 'Homing platform': {homing_position}")
print(f"Encoder value at the next experiment event: {next_event_position}")
print("❗ Warning: home position is determined by the time of the experiment event after 'Homing platform'. It works for e.g. 'Waiting for run threshold' which starts immediately after homing, but may not work for other session types.")

# Perform unit conversions if not already done
if not unit_conversions:
    harp_streams["OpticalTrackingRead0X(46)"] = process.running_unit_conversion(
        harp_streams["OpticalTrackingRead0X(46)"].to_numpy())  # m / s
    harp_streams["OpticalTrackingRead0Y(46)"] = process.turning_unit_conversion(
        harp_streams["OpticalTrackingRead0Y(46)"].to_numpy())  # degrees / s
    harp_streams["OpticalTrackingRead1X(46)"] = process.running_unit_conversion(
        harp_streams["OpticalTrackingRead1X(46)"].to_numpy())
    harp_streams["OpticalTrackingRead1Y(46)"] = process.turning_unit_conversion(
        harp_streams["OpticalTrackingRead1Y(46)"].to_numpy())
    
    ## Forward fill then bacward fill values to remove NaNs for optical sensors after conversion 
    # columns_to_fill = [
    #     "OpticalTrackingRead0X(46)", "OpticalTrackingRead0Y(46)",
    #     "OpticalTrackingRead1X(46)", "OpticalTrackingRead1Y(46)"]
    # harp_streams[columns_to_fill] = harp_streams[columns_to_fill].ffill().bfill()
    
    harp_streams["Encoder(38)"] = harp_streams["Encoder(38)"].ffill().bfill() #fill before unit conversion
    harp_streams["Encoder(38)"] = process.encoder_unit_conversion(
        harp_streams["Encoder(38)"], next_event_position)
    


    unit_conversions = True
    print("✅ Unit conversions to real-life values done")
else:
    print("❗ Flow sensor and encoder values already converted to real-world units, skipping")


In [ ]:
# calculate position from X and Y optical signals 
# only processing sensor 0 at this time 

columns_to_process = ["OpticalTrackingRead0X(46)", "OpticalTrackingRead0Y(46)"] # for sensor 1, add "OpticalTrackingRead1X(46)", "OpticalTrackingRead1Y(46)"
for column in columns_to_process:
    # Drop missing values
    optical_tracking_values = harp_streams[column].dropna().to_numpy()
    time_values_seconds = (harp_streams[column].dropna().index - harp_streams.index[0]).total_seconds()

    # Smooth the optical tracking values
    optical_tracking_values = process.moving_average_smoothing(optical_tracking_values, 3)

    # Use the cumulative trapezoidal rule to calculate the integral
    cumulative_integral = cumulative_trapezoid(optical_tracking_values, x=time_values_seconds, initial=0)

    # Create a new DataFrame with the resulting integral values and the original datetime index
    integral_df = pd.DataFrame(cumulative_integral, index=harp_streams[column].dropna().index, columns=[f"CumulativeIntegral_{column}"])

    # Reindex the processed data back to the original datetime index
    integral_df = integral_df.reindex(harp_streams.index, fill_value=np.nan)
    optical_tracking_values_reindexed = pd.Series(optical_tracking_values, index=harp_streams[column].dropna().index).reindex(harp_streams.index, fill_value=np.nan)

    # Return the smoothed optical_tracking_values to the source column in harp_streams
    harp_streams[column] = optical_tracking_values_reindexed
    harp_streams[f"Position_{column}"] = integral_df[f"CumulativeIntegral_{column}"]


In [ ]:
#---------------------------
# Encoder axes colours and labels are still wrong 
#---------------------------

# Define the columns to process
columns_to_process = ["OpticalTrackingRead0X(46)", "OpticalTrackingRead0Y(46)", 
                     "OpticalTrackingRead0Brightness(46)", "Encoder(38)"]

# Create a single figure with 4 subplots (2x2 grid)
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=columns_to_process,
                   specs=[[{"secondary_y": True}, {"secondary_y": True}],
                          [{"secondary_y": False}, {"secondary_y": True}]])

# Define consistent colors
primary_color = 'black'
secondary_color = 'blue'

# Process each column and add to the appropriate subplot
for i, column in enumerate(columns_to_process):
    row = i // 2 + 1
    col = i % 2 + 1
    
    if i < 2 and f"Position_{column}" in harp_streams.columns:
        # For the first two plots, use dual y-axes
        # Add the original values trace on primary y-axis
        fig.add_trace(
            go.Scatter(
                x=harp_streams.index,
                y=harp_streams[column],
                mode='lines+markers',
                name=f'raw counts',  # Simplified legend name
                line=dict(color=primary_color),
                marker=dict(size=4),
                legendgroup=f'group{i}',
                legendgrouptitle_text=column
            ),
            row=row, col=col, secondary_y=False
        )
        
        # Add the position trace on secondary y-axis
        fig.add_trace(
            go.Scatter(
                x=harp_streams.index,
                y=harp_streams[f"Position_{column}"],
                mode='lines+markers',
                name=f'position',  # Simplified legend name
                line=dict(color=secondary_color),
                marker=dict(size=4),
                legendgroup=f'group{i}',
                legendgrouptitle_text=column
            ),
            row=row, col=col, secondary_y=True
        )
        
        # Set y-axis titles with matching colors
        if column == "OpticalTrackingRead0X(46)":
            fig.update_yaxes(title_text="Running displacement (raw counts)", title_font=dict(color=primary_color), 
                             tickfont=dict(color=primary_color), row=row, col=col, secondary_y=False)
            fig.update_yaxes(title_text="Running position (m)", title_font=dict(color=secondary_color), 
                             tickfont=dict(color=secondary_color), row=row, col=col, secondary_y=True)
        elif column == "OpticalTrackingRead0Y(46)":
            fig.update_yaxes(title_text="Turning displacement (raw counts)", title_font=dict(color=primary_color), 
                             tickfont=dict(color=primary_color), row=row, col=col, secondary_y=False)
            fig.update_yaxes(title_text="Turning position (degrees)", title_font=dict(color=secondary_color), 
                             tickfont=dict(color=secondary_color), row=row, col=col, secondary_y=True)
            
    else:
        # For the other plots
        if column == "Encoder(38)":
            fig.add_trace(
                go.Scatter(
                    x=harp_streams.index,
                    y=harp_streams[column],
                    mode='lines+markers',
                    name=f'Converted degrees',  # Simplified legend name
                    line=dict(color=secondary_color), 
                    marker=dict(size=4),
                    legendgroup=f'group{i}',
                    legendgrouptitle_text=column
                ),
                row=row, col=col
            )
        else:
            # For other plots, keep the original color
            fig.add_trace(
                go.Scatter(
                    x=harp_streams.index,
                    y=harp_streams[column],
                    mode='lines+markers',
                    name=f'Raw',  # Simplified legend name
                    line=dict(color=primary_color),
                    marker=dict(size=4),
                    legendgroup=f'group{i}',
                    legendgrouptitle_text=column
                ),
                row=row, col=col
            )
        
        # Add the position trace if it exists
        if f"Position_{column}" in harp_streams.columns:
            fig.add_trace(
                go.Scatter(
                    x=harp_streams.index,
                    y=harp_streams[f"Position_{column}"],
                    mode='lines+markers',
                    name=f'Position',  # Simplified legend name
                    line=dict(color=secondary_color),
                    marker=dict(size=4),
                    legendgroup=f'group{i}',
                    legendgrouptitle_text=column
                ),
                row=row, col=col
            )
        
        # For the encoder plot, add the original values on a secondary y-axis
        if column == "Encoder(38)" and "Encoder(38)" in harp_streams_orig.columns:
            fig.add_trace(
            go.Scatter(
                x=harp_streams_orig.index,
                y=harp_streams_orig["Encoder(38)"],
                mode='lines+markers',
                name='Original',  # Simplified legend name
                line=dict(color=primary_color), 
                marker=dict(size=4),
                legendgroup=f'group{i}',
                legendgrouptitle_text=column
            ),
            row=row, col=col, secondary_y=True
            )
            
            # Update secondary y-axis title to match trace color (black)
            fig.update_yaxes(title_text="Encoder (raw)", title_font=dict(color=primary_color),
                             tickfont=dict(color=primary_color), row=row, col=col, secondary_y=True)
        
        # Set appropriate y-axis titles based on the column with matching colors
        if column == "OpticalTrackingRead0Brightness(46)":
            fig.update_yaxes(title_text="Sensor Brightness (SQUAL)", title_font=dict(color=primary_color),
                             tickfont=dict(color=primary_color), row=row, col=col)
        else:  # Encoder(38)
            # Update primary y-axis title for encoder to match trace color (blue)
            fig.update_yaxes(title_text="Encoder (degrees)", title_font=dict(color=secondary_color),
                     tickfont=dict(color=secondary_color), row=row, col=col)

# Update layout for the entire figure
fig.update_layout(
    title="Sensor Readings",
    height=900,  # Larger height for better visibility
    width=1200,  # Larger width
)

# Add separate legends to each subplot
for i in range(1, 3):
    for j in range(1, 3):
        subplot_index = (i-1)*2 + j - 1
        fig.update_layout(**{
            f'legend{subplot_index+1}': dict(
                x=0.01 + (j-1)*0.5,  # Position legends in their respective subplot areas
                y=0.99 - (i-1)*0.5,
                xanchor='left',
                yanchor='top',
                bgcolor='rgba(255, 255, 255, 0.8)',
                bordercolor='rgba(0, 0, 0, 0.2)',
                borderwidth=1,
                tracegroupgap=5
            )
        })

# Configure legend to show only in its subplot area
for trace in fig.data:
    trace.update(showlegend=True)
    if hasattr(trace, 'legendgroup'):
        group_num = int(trace.legendgroup[-1])
        legend_num = group_num + 1
        trace.update(legend=f'legend{legend_num}')

fig.update_xaxes(title_text="Time")

# Open the figure in the default web browser
pio.show(fig, renderer='browser')
